<a href="https://colab.research.google.com/github/skinnytwinvale/Amazon_vine_analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-25 01:25:22--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.64MB/s    in 0.2s    

2022-07-25 01:25:22 (5.64 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-ChallengeD2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [6]:
# Confirm total_vote, helpful_votes, and star_rating are all integers
df

DataFrame[marketplace: string, customer_id: int, review_id: string, product_id: string, product_parent: int, product_title: string, product_category: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string]

In [7]:
# Filter furniture data frame on total_vote greater than or equal to 20
totalvotes_df = df.filter("total_votes>=20")
totalvotes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    7142190| R4PKAZRQJJX14|B00QZLVCU0|     210935604|Valve - DOTA 2 St...|     Video Games|          1|           21|         34|   N|                N|What store doesn'...|Who pays 4 dollar...| 2015-08-31|
|         US|    1085641|R2CI0Y288CC7E2|B00RHI62GY|     626589765|ONE PIECE Pirate ...|     Video Games|          1|    

In [8]:
helpful_50_df = totalvotes_df.filter("helpful_votes/total_votes>=.50")
helpful_50_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    7142190| R4PKAZRQJJX14|B00QZLVCU0|     210935604|Valve - DOTA 2 St...|     Video Games|          1|           21|         34|   N|                N|What store doesn'...|Who pays 4 dollar...| 2015-08-31|
|         US|    1085641|R2CI0Y288CC7E2|B00RHI62GY|     626589765|ONE PIECE Pirate ...|     Video Games|          1|    

In [9]:
print(helpful_50_df.count())

40565


In [10]:
vine_review_df = helpful_50_df.filter(helpful_50_df["vine"] == "Y")
vine_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48771843|R3KKUSGFZWSUIY|B00YXO5UXG|     555176890|Turtle Beach - Ea...|     Video Games|          5|           56|         63|   Y|                N|Quality Chat Headset|Not every situati...| 2015-08-04|
|         US|   53080186|R10FO5UKKVZBK2|B00XO041RQ|     238654494|PDP AG7 True Wire...|     Video Games|          3|    

In [11]:
novine_review_df = helpful_50_df.filter(helpful_50_df["vine"] == "N")
novine_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    7142190| R4PKAZRQJJX14|B00QZLVCU0|     210935604|Valve - DOTA 2 St...|     Video Games|          1|           21|         34|   N|                N|What store doesn'...|Who pays 4 dollar...| 2015-08-31|
|         US|    1085641|R2CI0Y288CC7E2|B00RHI62GY|     626589765|ONE PIECE Pirate ...|     Video Games|          1|    

In [12]:
from pyspark.sql.functions import count
# Total number of reviews for each dataframe from Step 3 and Step 4:
vine_review_count = vine_review_df.count()
print("Total Number of Reviews PAID and 'helpful': %f" % vine_review_count)

novine_review_count = novine_review_df.count()
print("Total Number of Reviews NON-PAID and 'helpful': %f" % novine_review_count)

Total Number of Reviews PAID and 'helpful': 94.000000
Total Number of Reviews NON-PAID and 'helpful': 40471.000000


In [13]:
# Number of 5-star Reviews for above Paid and Non-Paid dataframes
star5_vine_df = vine_review_df.filter(novine_review_df["star_rating"]=="5")
print("Total Number of 5-star Reviews PAID and 'helpful': %f" % star5_vine_df.count())


star5_novine_df = novine_review_df.filter(novine_review_df["star_rating"]=="5")
print("Total Number of 5-star Reviews NON-PAID and 'helpful': %f" % star5_novine_df.count())

Total Number of 5-star Reviews PAID and 'helpful': 48.000000
Total Number of 5-star Reviews NON-PAID and 'helpful': 15663.000000


In [14]:
# Percentage of 5-star Reviews for above Paid and Non-Paid dataframes
star5_vine_pct = (star5_vine_df.count()/vine_review_count)
print("Percent of 5-Star Furniture Reviews from PAID, 'helpful' dataset: %f" % star5_vine_pct)

star5_novine_pct = (star5_novine_df.count()/novine_review_count)
print("Percent of 5-Star Furniture Reviews from NON-PAID, 'helpful' dataset: %f" % star5_novine_pct)

Percent of 5-Star Furniture Reviews from PAID, 'helpful' dataset: 0.510638
Percent of 5-Star Furniture Reviews from NON-PAID, 'helpful' dataset: 0.387018


In [15]:
# Total number of ALL reviews
total_reviews_count = df.count()
print("Total Number of Furniture Reviews: %f" % total_reviews_count)

help_reviews_count = helpful_50_df.count()
print("Total Number of 'Helpful' Furniture Reviews: %f" % help_reviews_count)

Total Number of Furniture Reviews: 1785997.000000
Total Number of 'Helpful' Furniture Reviews: 40565.000000


In [16]:
#Total number of ALL 5-star reviews
star5_df = df.filter(df["star_rating"] == '5')
star5_df.show()

star5_count = star5_df.count()
print("Total Number of 5-Star Furniture Reviews: %f" % star5_count)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [17]:

#Total number of 5-star HELPFUL reviews
star5_help_df = helpful_50_df.filter(helpful_50_df["star_rating"] == '5')
star5_help_df.show()

star5_help_count = star5_help_df.count()
print("Total Number of 5-Star 'Helpful' Furniture Reviews: %f" % star5_help_count)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   52553659|R36O341WWXXKNP|B00ZSJU97C|     838780144|Disney Infinity 3...|     Video Games|          5|           28|         31|   N|                N|Upgrade to Disney...|We have been play...| 2015-08-30|
|         US|   14073759|R29BOS5HMAY1LO|B00W435BL4|     281495895|Madden NFL 16 - P...|     Video Games|          5|    

In [18]:
# Percentage of ALL 5-star reviews paid vs non-paid 
# 5-star Paid reviews 
vine_star5_all_df = star5_df.filter(star5_df["vine"] == "Y")
vine_star5_all_count = vine_star5_all_df.count()

vine_star5_all_pct = (vine_star5_all_count/star5_count)
print("Number of 5-Star Furniture Reviews PAID: %f" % vine_star5_all_count)
print("Percent of 5-Star Furniture Reviews PAID: %f" % vine_star5_all_pct)
 
#5-star Non-Paid reviews
novine_star5_all_df = star5_df.filter(star5_df["vine"] == "N")
novine_star5_all_count = novine_star5_all_df.count()

novine_star5_all_pct = novine_star5_all_count/star5_count
print("Number of 5-Star Furniture Reviews NON-PAID: %f" % novine_star5_all_count)
print("Percent of 5-Star Furniture Reviews NON-PAID: %f" % novine_star5_all_pct)

Number of 5-Star Furniture Reviews PAID: 1607.000000
Percent of 5-Star Furniture Reviews PAID: 0.001565
Number of 5-Star Furniture Reviews NON-PAID: 1025317.000000
Percent of 5-Star Furniture Reviews NON-PAID: 0.998435


In [19]:
# Percentage of HELPFUL 5-star reviews paid vs non-paid (% of 5 star reviews in Step 3 dataframe) 
# 5-star Paid "helpful" reviews 
vine_star5_help_df = star5_help_df.filter(star5_help_df["vine"] == "Y")
vine_star5_help_count = vine_star5_help_df.count()

vine_star5_help_pct = (vine_star5_help_count/star5_help_count)
print("Number of 5-Star, 'Helpful' Furniture Reviews PAID: %f" % vine_star5_help_count)
print("Percent of 5-Star,'Helpful'Furniture Reviews PAID: %f" % vine_star5_help_pct)
 
#5-star Non-Paid "helpful" reviews
novine_star5_help_df = star5_help_df.filter(star5_help_df["vine"] == "N")
novine_star5_help_count = novine_star5_help_df.count()

novine_star5_help_pct = novine_star5_help_count/star5_help_count
print("Number of 5-Star, 'Helpful' Furniture Reviews NON-PAID: %f" % novine_star5_help_count)
print("Percent of 5-Star,'Helpful'Furniture Reviews NON-PAID: %f" % novine_star5_help_pct)

Number of 5-Star, 'Helpful' Furniture Reviews PAID: 48.000000
Percent of 5-Star,'Helpful'Furniture Reviews PAID: 0.003055
Number of 5-Star, 'Helpful' Furniture Reviews NON-PAID: 15663.000000
Percent of 5-Star,'Helpful'Furniture Reviews NON-PAID: 0.996945


In [20]:
#Percent of Vine vs non-Vine all reviews
vine_all_df = df.filter(df["vine"] == "Y")
novine_all_df = df.filter(df["vine"] == "N")
vine_pct_all = (vine_all_df.count()/df.count())
novine_pct_all = (novine_all_df.count()/df.count())


print(df.count())
print(vine_all_df.count())
print(novine_all_df.count())
print(vine_pct_all)
print(novine_pct_all)

1785997
4291
1781706
0.0024025796235939926
0.997597420376406


In [21]:
# Percent of All Vine that is 5-Star
vine_all_star5_df = vine_all_df.filter(vine_all_df["star_rating"] == '5')
vine_all_star_df = vine_all_df.filter(vine_all_df["star_rating"] != '5')

novine_all_star5_df = novine_all_df.filter(novine_all_df["star_rating"] == '5')
novine_all_star_df = novine_all_df.filter(novine_all_df["star_rating"] != '5')

vine_pct_star5_all = (vine_all_star5_df.count()/vine_all_df.count())
novine_pct_star5_all = (novine_all_star5_df.count()/novine_all_df.count())

print(vine_all_star5_df.count())
print(vine_all_star_df.count())

print(novine_all_star5_df.count())
print(novine_all_star_df.count())

print(vine_pct_star5_all)
print(novine_pct_star5_all)

1607
2684
1025317
756389
0.3745047774411559
0.5754692412777417


In [22]:
# Pull out all Vine reviewed products
vine_prod_df = vine_all_df.select(vine_all_df["product_id"]).dropDuplicates()
vine_prod_df.show()
vine_prod_df.count()

+----------+
|product_id|
+----------+
|B00BMFIXT2|
|B006WQR3GA|
|B00CBVHI5Q|
|B0057FANEQ|
|B006VE44A6|
|B008J16AG0|
|B00YY3P4RI|
|B005NH6NMY|
|B00NJH1EO2|
|B00HRKE1SI|
|B008VQ68C4|
|B003ULODDG|
|B00NY5YYT6|
|B00EDNLO32|
|B00EDNO5WY|
|B0050SVGMS|
|B00BGD6LMG|
|B005JVKAY6|
|B00U33Q940|
|B00KAED850|
+----------+
only showing top 20 rows



238

In [23]:
# create an inner join between all the Vine Reviews df and the full df
inner_join = vine_prod_df.join(df, vine_all_df.product_id == df.product_id, how="inner").dropDuplicates()
inner_join.show()
inner_join.count()

+----------+-----------+-----------+--------------+----------+--------------+-------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|product_id|marketplace|customer_id|     review_id|product_id|product_parent|      product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+----------+-----------+-----------+--------------+----------+--------------+-------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|B006WQR3GA|         US|   41488105|R31F8WAFV7TOIX|B006WQR3GA|      66148347|XCOM: Enemy Unknown|     Video Games|          3|            0|          0|   N|                Y|         Three Stars|Game was in good ...| 2015-08-26|
|B006WQR3GA|         US|   42912521|R26L3RWN1A9W4Q|B006WQR3GA|      66148347|XCO

47427

In [24]:
# Rating breakdown 
new_df = inner_join.groupby("star_rating").count()
new_df.show()

+-----------+-----+
|star_rating|count|
+-----------+-----+
|          1| 3812|
|          3| 4171|
|          5|27741|
|          4| 9182|
|          2| 2521|
+-----------+-----+



In [25]:
# Total Vine Reviews for Vine Products
vine_new_df = inner_join.filter(inner_join["vine"] == "Y")
vine_new_df.count()

4291

In [26]:
# Total Vine Reviews for Vine Products
novine_new_df = inner_join.filter(inner_join["vine"] == "N")
novine_new_df.count()

43136

In [27]:
# % of 5-star reviews are Vine or non-Vine Reviews
vine_star5_new = vine_new_df.groupby("star_rating").count()
vine_star5_new.show()

+-----------+-----+
|star_rating|count|
+-----------+-----+
|          1|   60|
|          3|  718|
|          5| 1607|
|          4| 1712|
|          2|  194|
+-----------+-----+



In [28]:
# % of 5-star reviews are Vine or non-Vine Reviews
novine_star5_new = novine_new_df.groupby("star_rating").count()
novine_star5_new.show()

+-----------+-----+
|star_rating|count|
+-----------+-----+
|          1| 3752|
|          3| 3453|
|          5|26134|
|          4| 7470|
|          2| 2327|
+-----------+-----+

